In [ ]:
# pip install cinemagoer

# Notebook to fetch the top100 actors and actressess from IMDB top list
1. It will save the output into a csv file (list_act.csv), containing the rank, name, imdb_id
2. It will retrieve images from imdb and save them to the output folder

In [2]:
#Webscrapping from IMDB
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import imdb
from imdb import helpers as hlp
import requests
import os
import unicodedata


In [ ]:

# url list of top 100 actresses and actors from IMDB
url_list = ["https://www.imdb.com/list/ls063784435/", "https://www.imdb.com/list/ls050274118/"] #url for top 100 actors and top100 actressess

# url_list = ["https://www.imdb.com/list/ls058011111/?page=1"] #1000 actors and actresses 10 pages

# dictionary with the keys that will be used to store the data in a csv file
act_dict = {
    "rank":[],
    "name":[],
    "imdb_id":[]
    }

for url in url_list:
    response = get(url) 

    #parse the content of current iteration of request
    page_html = BeautifulSoup(response.text, 'html.parser')
        
    movie_containers = page_html.find_all('div', class_ = 'lister-item mode-detail')

    for container in movie_containers:
        act_dict["rank"].append(container.find('h3', class_ = 'lister-item-header').find('span', class_ = 'lister-item-index unbold text-primary').text.split(".")[0])
        act_dict["name"].append(container.find('h3', class_ = 'lister-item-header').find("a").text.strip())
        act_dict["imdb_id"].append(container.find('h3', class_ = 'lister-item-header').find("a")["href"].split("/")[2])
   
df_act = pd.DataFrame(act_dict)
df_act.to_csv("../raw_data/list_act.csv")


In [3]:
# Read the list_act.csv file to a dataframe
df_act = pd.read_csv("../raw_data/list_act.csv")


### 2. From the saved csv retrieve the images from imdb and save them to the target folder

In [4]:
ia = imdb.Cinemagoer()

# take out accent's from name
def strip_accents(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

for actor in df_act["imdb_id"]:
    actor_id = int(actor.strip("nm"))
    person = ia.get_person(actor_id)
    jpg_path = hlp.resizeImage(person["headshot"],crop="0.2")

    person_name = "_".join(strip_accents(person["name"].lower()).split())+".jpg"

    target_path = os.path.join(os.getcwd(),"..","raw_data","output_imdb_top100",person_name)

    original = jpg_path
    target = target_path

    url = jpg_path
    
    # This statement requests the resource at
    # the given link, extracts its contents
    # and saves it in a variable
    data = requests.get(url).content
    
    # Opening a new file named img with extension .jpg
    # This file would store the data of the image file
    f = open(target_path,'wb')
    
    # Storing the image data inside the data variable to the file
    f.write(data)
    f.close()
 

2023-06-21 11:01:56,191 CRITICAL [imdbpy] /home/jal/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/imdb/_exceptions.py:32: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/name/nm0266422/fullcredits', 'proxy': '', 'exception type': 'IOError', 'original exception': TimeoutError('The read operation timed out')},); kwds: {}
Traceback (most recent call last):
  File "/home/jal/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/imdb/parser/http/__init__.py", line 233, in retrieve_unicode
    response = uopener.open(url)
  File "/home/jal/.pyenv/versions/3.10.6/lib/python3.10/urllib/request.py", line 519, in open
    response = self._open(req, data)
  File "/home/jal/.pyenv/versions/3.10.6/lib/python3.10/urllib/request.py", line 536, in _open
    result = self._call_chain(self.handle_open, protocol, protocol +
  File "/home/jal/.pyenv/versions/3.10.6/lib/python3.10/urllib/request.py", line 496, in _c

IMDbDataAccessError: {'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/name/nm0266422/fullcredits', 'proxy': '', 'exception type': 'IOError', 'original exception': TimeoutError('The read operation timed out')}